# Ray Serve - Model Serving Challenges

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

## The Challenges of Model Serving

Model development happens in a data science research environment. There are many challenges, but also tools at the data scientists disposal.

Model deployment to production faces an entirely different set of challenges and requires different tools, although it is desirable to bridge the divide as much as possible.

Here is a partial lists of the challenges of model serving:

### It Should Be Framework Agnostic

Model serving frameworks must be able to serve models from popular systems like TensorFlow, PyTorch, scikit-learn, or even arbitrary Python functions. Even within the same organization, it is common to use several machine learning frameworks. 

Also, machine learning models are typically surrounded by lots of application logic. For example, some model serving is implemented as a RESTful service to which scoring requests are made. Often this is too restrictive, as some additional processing may be desired as part of the scoring process, and the performance overhead of remote calls may be suboptimal.

### Pure Python

It has been common recently for model serving to be done using JVM-based systems, since many production enterprises are JVM-based. This is a disadvantage when model training and other data processing are done using Python tools, only. 

In general, model serving should be intuitive for developers and simple to configure and run. Hence, it is desirable to use pure Python and to avoid verbose configurations using YAML files or other means. 

Data scientists and engineers use Python to develop their machine learning models, so they should also be able to use Python to deploy their machine learning applications. This need is growing more critical as online learning applications combine training and serving in the same applications.

### Simple and Scalable

Model serving must be simple to scale on demand across many machines. It must also be easy to upgrade models dynamically, over time. Achieving production uptime and performance requirements are essential for success.

### DevOps Integrations

Model serving deployments need to integrate with existing "DevOps" CI/CD practices for controlled, audited, and predicatble releases. Patterns like [Canary Deployment](https://martinfowler.com/bliki/CanaryRelease.html) are particularly useful for testing the efficacy of a new model before replacing existing models, just as this pattern is useful for other software deployments.

### Flexible Deployment Patterns

There are unique deployment patterns, too. For example, it should be easy to deploy a forest of models, to split traffic to different instances, and to score data in batches for greater efficiency.

See also this [Ray blog post](https://medium.com/distributed-computing-with-ray/the-simplest-way-to-serve-your-nlp-model-in-production-with-pure-python-d42b6a97ad55) on the challenges of model serving and the way Ray Serve addresses them. It also provides an example of starting with a simple model, then deploying a more sophisticated model into the running application.

## Why Ray Serve?

[Ray Serve](https://docs.ray.io/en/latest/serve/index.html) is a scalable model-serving library built on [Ray](https://ray.io).

For users, Ray Serve offers these benefits:

* **Framework Agnostic**: You can use the same toolkit to serve everything from deep learning models built with [PyTorch](https://docs.ray.io/en/latest/serve/tutorials/pytorch.html#serve-pytorch-tutorial), [Tensorflow](https://docs.ray.io/en/latest/serve/tutorials/tensorflow.html#serve-tensorflow-tutorial), or [Keras](https://docs.ray.io/en/latest/serve/tutorials/tensorflow.html#serve-tensorflow-tutorial), to [scikit-Learn](https://docs.ray.io/en/latest/serve/tutorials/sklearn.html#serve-sklearn-tutorial) models, to arbitrary business logic.
* **Python First:** Configure your model serving with pure Python code. No YAML or JSON configurations required.

As a library, Ray Serve enables the following:

* [Splitting traffic between backends dynamically](https://docs.ray.io/en/latest/serve/advanced.html#serve-split-traffic) with zero downtime. This is accomplished by decoupling routing logic from response handling logic.
* [Support for batching](https://docs.ray.io/en/latest/serve/advanced.html#serve-batching) to improve performance helps you meet your performance objectives. You can also use a model for batch and online processing.
* Because Serve is a library, it's esay to integrate it with other tools in your environment, such as CI/CD.

Since Serve is built on Ray, it also allows you to scale to many machines, in your datacenter or in cloud environments, and it allows you to leverage all of the other Ray frameworks.

## Two Simple Ray Serve Examples

We'll explore a more detailed example in the next lesson, where we actually serve ML models. Here we explore how simple deployments are simple with Ray Serve! We will first use a function that does "scoring", sufficient for _stateless_ scenarios, then a use class, which enables _stateful_ scenarios.

But first, initialize Ray as before:

In [ ]:
import ray
from ray import serve

import requests  # for making web requests

In [ ]:
ray.init(ignore_reinit_error=True)

Now we initialize Serve itself:

In [ ]:
serve.init(name='serve-example-1')  # Name for this Serve instance

Next, define our stateless function for processing requests.

Note that Serve leverages the [Flask API](https://flask.palletsprojects.com/en/1.1.x/api/), which is often familiar to developers, as it is a natural first approach for deploying models as RESTful services.

In [ ]:
def echo(flask_request):          # Uses the Flask API 
    return "hello " + flask_request.args.get("name", "serve!")

In [ ]:
serve.create_backend("hello", echo)
serve.create_endpoint("hello", backend="hello", route="/hello")

In [ ]:
for i in range(10):
    response = requests.get(f"http://127.0.0.1:8000/hello?name=request_{i}").text
    print(f'{i:2d}: {response}')

You should see `hello request_N` in the output. Try making `requests.get()` invocations without the `?name=request_{i}` parameter. You should see `hello serve!`.

We'll explain the concepts of _backends_ and _endpoints_ below. 

Now let's serve another "model" in the same service:

In [ ]:
class Counter:
    def __init__(self, initial_count = 0):
        self.count = initial_count

    def __call__(self, flask_request):
        self.count += 1
        return {"current_counter": self.count, "args": flask_request.args}

When we create the _backend_, we can pass constructor arguments after the label and the name of the class:

In [ ]:
serve.create_backend("counter", Counter, 0)  # initial_count = 0
serve.create_endpoint("counter", backend="counter", route="/counter")

In [ ]:
for i in range(10):
    response = requests.get(f"http://127.0.0.1:8000/counter?i={i}").json()
    print(f'{i:2d}: {response}')

## Exercise - Add Another New Backend and Endpoint

Using either a function or a stateful class, add another _backend_ and _endpoint_, then try it out.

## Ray Serve Concepts

Let's explain _backends_ and _endpoints_.

For more details, see this [key concepts](https://docs.ray.io/en/latest/serve/key-concepts.html) documentation.

### Backends

Backends define the implementation of your business logic or models that will handle requests when queries come in to _endpoints._ 

To define a backend, first define the “handler” or business logic that will take requests and construct responses. Specifically, the handler should take as input a [Flask Request object](https://flask.palletsprojects.com/en/1.1.x/api/?highlight=request#flask.Request) and return any JSON-serializable object as output. 

Use a function when your response is _stateless_ and a class when your response is _stateful_ (although the class instances could be stateless, of course). Another advantage of using a class is the ability to specify constructor arguments in `serve.create_backend`, as was shown in the `counter` example above.

Finally, a backend is defined using `serve.create_backend`, specifying a logical, unique name, and the handler.

You can list all defined backends and delete them to reclaim resources. However, a backend cannot be deleted while it is in use by an endpoint, because then traffic to an endpoint could not be handled:

In [ ]:
serve.create_backend("counter_toss", Counter, 0)
serve.list_backends()

In [ ]:
serve.delete_backend("counter_toss")

In [ ]:
serve.list_backends()

### Endpoints

While a backend defines the request handling logic, an endpoint allows you to expose a backend via HTTP. Endpoints are “logical” and can have one or multiple backends that serve requests to them. 

To create an endpoint, you specify a name for the endpoint, the name of a backend to handle requests to the endpoint, and the route and the list of HTTP methods (e.g., `[GET]`, which is the default) where it will be accesible. By default endpoints are serviced only by the backend provided to `serve.create_endpoint`, but in some cases you may want to specify multiple backends for an endpoint, e.g., for A/B testing or incremental rollout. For information on traffic splitting, please see [Splitting Traffic Between Backends](https://docs.ray.io/en/latest/serve/advanced.html#serve-split-traffic).

Let's define a second endpoint for our `hello` backend, this one providing `POST` access. (We could have defined the original `hello` endpoint to support `POST` and `GET` using `methods = ['POST', 'GET']`.)

In [ ]:
serve.create_endpoint("post_hello", backend="hello", route="/post_hello", methods=["POST"])

In [ ]:
eds = serve.list_endpoints()
eds.keys(), eds

In [ ]:
for i in range(10):
    response = requests.post(f"http://127.0.0.1:8000/post_hello", data = {'name': f'request_{i}'}).text
    print(f'{i:2d}: {response}')

In this case, the data is not part of the `args` that our function expects to find, so the default `serve!` is returned.

### Splitting Traffic Between Backends

There are [several more advanced customizations](https://docs.ray.io/en/latest/serve/advanced.html) you can do. Let's look at a common one that supports the model deployment patterns we discussed above; splitting traffic between different backends. 

We'll implement the [Canary Deployment](https://martinfowler.com/bliki/CanaryRelease.html) pattern for testing a new model.

First, let's reuse our original `echo` function as the "old" backend and define a new backend function, `new_echo`, which will handle `POST` requests better. Recall that the `echo` backend was named `hello`.

In [ ]:
def new_echo(flask_request):          # Uses the Flask API 
    name = flask_request.args.get("name", None) or flask_request.form.get("name", "serve!")
    return "hello " + name + " (new)"

In [ ]:
serve.create_backend("new_hello", new_echo)

Initially, set all traffic to be served by the "old" backend. Note that our endpoint handles both `GET` and `POST` requests.

In [ ]:
serve.create_endpoint("canary_endpoint", backend="hello", route="/canary-test", methods=['GET', 'POST'])

Normally, you would only direct 1% or less of the traffic to the new backend in a real Canary deployment, but we'll use 25% so we see a lot of "hits". Now define the traffic split between the two backends:

In [ ]:
serve.set_traffic("canary_endpoint", {"hello": 0.75, "new_hello": 0.25})

In [ ]:
for i in range(10):
    gresponse = requests.get(f"http://127.0.0.1:8000/canary-test?name='request_{i}").text
    presponse = requests.post(f"http://127.0.0.1:8000/canary-test", data = {'name': f'request_{i}'}).text
    print(f'{i:2d}: GET: {gresponse:25s} POST: {presponse}')

The "new" implementation correctly handles data for `POST` requests. The old implementation always returns `serve!`, while both work fine for `GET` requests. 

Now route all traffic to the new, better backend.

In [ ]:
serve.set_traffic("canary_endpoint", {"new_hello": 1.0})

In [ ]:
for i in range(10):
    gresponse = requests.get(f"http://127.0.0.1:8000/canary-test?name='request_{i}").text
    presponse = requests.post(f"http://127.0.0.1:8000/canary-test", data = {'name': f'request_{i}'}).text
    print(f'{i:2d}: GET: {gresponse:25s} POST: {presponse}')

By the way, notice that this example really has nothing to do with model serving, per se. We basically have a flexible framework for any kind of request-response serving, backed by the transparent ability to scale to a cluster with Ray. 

Recently, a new `shadow_traffic` method was added, `serve.shadow_traffic(endpoint, backend, fraction)`. It allows you to duplicate a fraction of the traffic to another backend. This is useful for sampling and for testing a new algorithm without affecting current handling.

## Serve is a Singleton in the Ray Cluster

You may have noticed that when defining endpoints and backends, we called Serve API methods, not methods on a Serve _class instance_. Serve defaults to being a [singleton](https://en.wikipedia.org/wiki/Singleton_pattern) in the whole Ray cluster, not just the driver program. We passed a name argument to `serve.init()`, which creates a separate Ray actor internally, but the endpoints and backends defined are still global.

This means that even when you terminate this notebook, our definitions above will persist! Hence, you need to clean up any endpoints and backends that are no longer needed or shutdown Serve completely.

If you don't want to shut down serve, but remove everything currently defined, the following statements can be used:

In [ ]:
eps = serve.list_endpoints()
for name in eps.keys():
    serve.delete_endpoint(name)

In [ ]:
bes = serve.list_backends()
for name in bes.keys():
    serve.delete_backend(name)

In [ ]:
eps = serve.list_endpoints()
bes = serve.list_backends()
print(f'endpoints: {eps}, backends {bes}')

The previous steps aren't necessary if you want to shutdown Serve completely. Just run the following: 

In [ ]:
serve.shutdown()

In [ ]:
ray.shutdown()  # "Undo ray.init()".